In [2]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

# --- PARAMETERS ---
n = 10
b = 0.005  # log volatility per node
q = 0.5    # branching probability
face_value = 100

# --- MARKET DATA ---
spot_rates_pct = np.array([3.0, 3.1, 3.2, 3.3, 3.4, 3.5, 3.55, 3.6, 3.65, 3.7])
spot_rates = spot_rates_pct / 100.0
Z_market = face_value / (1 + spot_rates) ** np.arange(1, n + 1)

# --- BDT SHORT RATE LATTICE using r_ij = a_i * exp(b * j) ---
def build_short_rate_lattice(a, b, n):
    r = np.zeros((n, n))
    for i in range(n):
        for j in range(i + 1):
            r[i, j] = a[i] * np.exp(b * j)
    return r

# --- ELEMENTARY PRICE LATTICE Q ---
def build_elementary_price_lattice(r, q=0.5):
    Q = np.zeros((n + 1, n + 1))
    Q[0, 0] = 1.0
    for i in range(n):
        for j in range(i + 1):
            disc = 1 / (1 + r[i, j])
            Q[i + 1, j] += q * Q[i, j] * disc
            Q[i + 1, j + 1] += (1 - q) * Q[i, j] * disc
    return Q

# --- ZCB PRICES FROM BDT ---
def compute_bdt_ZCB_prices(a):
    r = build_short_rate_lattice(a, b, n)
    Q = build_elementary_price_lattice(r, q)
    Z_bdt = np.array([np.sum(Q[i + 1, :i + 2]) * face_value for i in range(n)])
    return Z_bdt, r, Q

# --- CALIBRATION OBJECTIVE (based on spot rate error) ---
def calibration_objective(a):
    Z_bdt, _, _ = compute_bdt_ZCB_prices(a)
    spot_bdt = (face_value / Z_bdt) ** (1 / np.arange(1, n + 1)) - 1
    return np.sum((spot_bdt - spot_rates) ** 2)

# --- CALIBRATION USING L-BFGS-B with bounds ---
a_initial = 1 + spot_rates
bounds = [(0.0001, 0.2)] * n

result = minimize(
    calibration_objective,
    a_initial,
    method='L-BFGS-B',
    bounds=bounds,
    options={'ftol': 1e-12, 'disp': True, 'maxiter': 10000}
)
a_optimized = result.x

# --- FINAL LATTICES ---
Z_bdt_final, r_final, Q_final = compute_bdt_ZCB_prices(a_optimized)
spot_bdt_final = (face_value / Z_bdt_final) ** (1 / np.arange(1, n + 1)) - 1
squared_diffs = (spot_bdt_final - spot_rates) ** 2

# --- DATAFRAMES ---
r_df_opt = pd.DataFrame(r_final).round(6)
Q_df_opt = pd.DataFrame(Q_final).round(6)
Z_market_df = pd.Series(Z_market).round(6)
Z_bdt_df = pd.Series(Z_bdt_final).round(6)
spot_bdt_df = pd.Series(spot_bdt_final).round(6)
squared_diffs_df = pd.Series(squared_diffs).round(12)

# --- PAYER SWAPTION PRICING ---
notional = 1_000_000
K = 0.039
expiry = 3
swap_start = 4
swap_end = 10

swap_values = np.zeros(expiry + 1)
for j in range(expiry + 1):
    swap_value = 0
    for t in range(swap_start, swap_end + 1):
        if j <= t - 1:
            r_ij = r_df_opt.iloc[t - 1, j]
            discount_factor = 1 / (1 + r_ij) ** (t - expiry)
            swap_value += K * notional * discount_factor
    swap_values[j] = max(swap_value - notional, 0)

swaption_price = sum(Q_df_opt.iloc[expiry, j] * swap_values[j] for j in range(expiry + 1))
swaption_price_rounded = round(swaption_price)

# --- FINAL OUTPUT ---
{
    "Calibrated a_i": np.round(a_optimized, 6).tolist(),
    "BDT ZCB Prices": Z_bdt_df.tolist(),
    "Market ZCB Prices": Z_market_df.tolist(),
    "Spot Rates from BDT": spot_bdt_df.tolist(),
    "Squared Differences": squared_diffs_df.tolist(),
    "Total Squared Error": float(np.sum(squared_diffs)),
    "Payer Swaption Price (Rounded)": swaption_price_rounded
}


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           10     M =           10

At X0        10 variables are exactly at the bounds

At iterate    0    f=  2.79353D-01    |proj g|=  1.99900D-01

At iterate    1    f=  7.71301D-03    |proj g|=  1.68414D-01

At iterate    2    f=  9.89526D-04    |proj g|=  8.44259D-03

At iterate    3    f=  8.99394D-04    |proj g|=  7.82708D-03

At iterate    4    f=  4.76877D-04    |proj g|=  8.92042D-03

At iterate    5    f=  3.16184D-04    |proj g|=  5.56307D-03

At iterate    6    f=  1.61832D-04    |proj g|=  3.25100D-03

At iterate    7    f=  7.89980D-05    |proj g|=  2.93247D-03

At iterate    8    f=  4.33213D-05    |proj g|=  1.18173D-03

At iterate    9    f=  2.89837D-05    |proj g|=  8.81119D-04

At iterate   10    f=  1.46432D-05    |proj g|=  5.76344D-04

At iterate   11    f=  8.31887D-06    |proj g|=  3.92921D-04

At iterate   12    f=  4.43473D-06    |proj g|=  3.26225D-04

At iterate   13    f=  4.1

{'Calibrated a_i': [0.030001,
  0.031922,
  0.033833,
  0.035796,
  0.037469,
  0.039644,
  0.037984,
  0.038851,
  0.039526,
  0.040504],
 'BDT ZCB Prices': [97.087303,
  94.076701,
  90.983057,
  87.815921,
  84.613631,
  81.348028,
  78.327787,
  75.34872,
  72.428013,
  69.546942],
 'Market ZCB Prices': [97.087379,
  94.076829,
  90.983137,
  87.821068,
  84.605249,
  81.350064,
  78.333815,
  75.356715,
  72.422955,
  69.536437],
 'Spot Rates from BDT': [0.030001,
  0.031001,
  0.032,
  0.033015,
  0.03398,
  0.035004,
  0.035511,
  0.036014,
  0.036492,
  0.036984],
 'Squared Differences': [1e-12,
  0.0,
  0.0,
  2.29e-10,
  4.2e-10,
  1.9e-11,
  1.3e-10,
  1.89e-10,
  6.5e-11,
  2.45e-10],
 'Total Squared Error': 1.2972038477374795e-09,
 'Payer Swaption Price (Rounded)': 0}